<a href="https://colab.research.google.com/github/carmeniturbe/ejercicios_adv_machine_learning/blob/main/03_Ejercicio_de_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tarea
La tarea es realizar ACP para acelerar un algoritmo de clasificación en un conjunto de datos de alta dimensión.  Ajustarás un modelo en los datos escalados originales, y uno diferente en los datos después de transformarlos utilizando un modelo ACP.  Compararás el tiempo de cálculo y las puntuaciones de evaluación.



Utilizaremos los dígitos del conjunto de datos de MNIST, que viene preinstalado en sklearn. Este conjunto de datos tiene imágenes de 28x28 píxeles de dígitos escritos a mano del 0 al 9. La tarea es clasificarlos para determinar qué dígitos son.

Utiliza ACP para disminuir las dimensiones en este conjunto de datos a medida que se retiene el 95 % de la varianza.  Puedes hacer esto cuando instancies el ACP al dar el argumento “n_components=” un número flotante entre 0 y 1.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import make_column_transformer, make_column_selector

In [2]:
# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
# view the shape of the dataset
mnist.data.shape


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)

El conjunto de datos tiene forma (70000, 784), lo que significa que estamos trabajando con 70 000 imágenes con 784 dimensiones.

Ten en cuenta que puedes acceder a los datos X utilizando “mnist.data” y acceder al objetivo utilizando “mnist.target”.

Si te aparece un error utilizando el código anterior, también puedes cargar los datos utilizando:

In [3]:
from keras.datasets import mnist
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape data
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

11490434/11490434 [==============================] - 0s 0us/step


2. Prepara los datos
Prepara los datos para el modelado.  Escala y aplica ACP a tus datos mientras se retiene el 95 % de la varianza.  Asegúrate de no filtrar ninguna información.

In [4]:
len(y_train)

60000

In [5]:
X_train.shape

(60000, 784)

In [6]:
len(y_test)

10000

In [7]:
X_test.shape

(10000, 784)

In [8]:
# Convertimos a dataframe para visualizar mejor
X_train = pd.DataFrame(X_train)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Scaler
scaler = StandardScaler()

# Create a transformer pipeline with PCA
PCA_pipeline = make_pipeline(scaler, PCA(n_components=.95), KNeighborsClassifier(n_neighbors=5))

# Fit and evaluate the PCA model
PCA_pipeline.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.95)),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [10]:
# Create a StandardScaler for the original data
original_data_scaler = StandardScaler()

# Create a KNeighborsClassifier pipeline for the original data
knn_original_pipeline = make_pipeline(original_data_scaler, KNeighborsClassifier(n_neighbors=5))

# Fit and evaluate the KNeighborsClassifier on the original data
knn_original_pipeline.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [11]:
%%time
pca_train_accuracy = PCA_pipeline.score(X_train, y_train)
pca_test_accuracy = PCA_pipeline.score(X_test, y_test)

print('PCA Model - Training Accuracy:', pca_train_accuracy)
print('PCA Model - Testing Accuracy:', pca_test_accuracy)

PCA Model - Training Accuracy: 0.9675833333333334
PCA Model - Testing Accuracy: 0.9486
CPU times: user 3min 19s, sys: 909 ms, total: 3min 20s
Wall time: 2min 11s


In [12]:
%%time
knn_original_train_accuracy = knn_original_pipeline.score(X_train, y_train)
knn_original_test_accuracy = knn_original_pipeline.score(X_test, y_test)

print('KNeighborsClassifier - Training Accuracy (Original Data):', knn_original_train_accuracy)
print('KNeighborsClassifier - Testing Accuracy (Original Data):', knn_original_test_accuracy)

KNeighborsClassifier - Training Accuracy (Original Data): 0.96425
KNeighborsClassifier - Testing Accuracy (Original Data): 0.9443
CPU times: user 7min 11s, sys: 801 ms, total: 7min 12s
Wall time: 4min 37s


We achieved similar results with both models; however, there was a notable difference in processing time. The model using the original data took twice as long to compute compared to the PCA-based model. Considering that the PCA model retains 95% of the variance, which is considered a robust representation, it stands out as the preferred choice. It not only yielded nearly identical accuracy scores but also accomplished this in half the time.

PCA Model:

Training Accuracy: 0.9676
Testing Accuracy: 0.9486
Computation Time (CPU): 3min 20s
Computation Time (Wall): 2min 11s
Original Data Model:

Training Accuracy: 0.9643
Testing Accuracy: 0.9443
Computation Time (CPU): 7min 12s
Computation Time (Wall): 4min 37s

The PCA model's efficiency and comparable performance make it the preferred option for our dataset.



The code below is WRONG! I left it only to remember that I have to use one scaler for each pipeline next time:

In this code I was using the same scaler for both and the machine got confused:

When you use the same scaler for both the PCA-transformed data and the original data, it's crucial to ensure that the dimensions (number of features) of both datasets match the expectations of the classifier.

The KNeighborsClassifier was expecting 784 features, which suggests that it might not have been aware of the dimensionality reduction applied by PCA. This can happen when the same scaler is used for both datasets, and the PCA transformation is not applied consistently.

By using separate scalers in the modified code, you ensure that the scaling is applied independently to each dataset. This means that the PCA-transformed data retains the reduced dimensionality introduced by PCA, while the original data is scaled appropriately. This separation prevents conflicts in the dimensionality and helps avoid the error you encountered.

Using separate scalers as in the modified code helps prevent such errors and provides more control over the preprocessing of each dataset, especially when you have dimensionality reduction techniques like PCA in your pipeline.

In [13]:
# Scaler
scaler = StandardScaler()
# Creamos el modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=5)

# Create a transformer pipeline with PCA
PCA_pipeline = make_pipeline(scaler, PCA(n_components=.95), knn_model)
PCA_pipeline.fit(X_train, y_train)  # Fit the pipeline to the training data

# Create a pipeline for the original df
knn_pipeline = make_pipeline(scaler, knn_model)
knn_pipeline.fit(X_train, y_train)  # Fit the pipeline to the training data

print('Training accuracy of PCA df:', PCA_pipeline.score(X_train, y_train))
print('Testing accuracy of PCA df:', PCA_pipeline.score(X_test, y_test))

print('Training accuracy of original df:', knn_pipeline.score(X_train, y_train))
print('Testing accuracy of original df:', knn_pipeline.score(X_test, y_test))

ValueError: ignored